In [1]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import random
import zipfile
device='cpu'

In [2]:
def load_data_jay_lyrics():
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars=f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([( char,i ) for i , char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [3]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

In [4]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size

In [15]:
def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return nn.Parameter(ts,requires_grad=True)
    
    def _three():
        return (_one((num_inputs,num_hiddens)),_one((num_hiddens,num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens,dtype=torch.float32),requires_grad=True))
    
    W_xz,W_hz,b_z=_three()
    W_xr,W_hr,b_r=_three()
    W_xh,W_hh,b_h=_three()
    
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,dtype=torch.float32),requires_grad=True)
    
    return nn.ParameterList([W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q])
        

In [16]:
def init_gru_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

In [29]:
def gru(inputs,state,params):
    W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q=params
    H,=state
    outputs=[]
    for X in inputs:
        Z=torch.sigmoid(torch.matmul(X,W_xz)+torch.matmul(H,W_hz)+b_z)
        R=torch.sigmoid(torch.matmul(X,W_xr)+torch.matmul(H,W_hr)+b_r)
        H_tilda=torch.tanh(torch.matmul(X,W_xh)+R*(torch.matmul(H,W_hh)+b_h))
        H=Z*H+(1-Z)*H_tilda
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)
    
    

In [30]:
um_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32,1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [31]:
def to_onehot(x,n_class):
    def one_hot(x,n_class):
        x=x.long()
        res=torch.zeros(x.shape[0],n_class,dtype=torch.float32)
        res=res.scatter(1,x.view(-1,1),1)
        return res
    return [one_hot(x[:,i],n_class) for i in range(x.shape[1])]

In [32]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.Tensor(corpus_indices)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        x=indices[:,i:i+num_steps]
        y=indices[:,i+1:i+num_steps+1]
        yield x,y

In [40]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state=init_rnn_state(1,num_hiddens,device)
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        x=to_onehot(torch.Tensor([[output[-1]]]),vocab_size)
        (y,state)=rnn(x,state,params)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [41]:
state=init_gru_state(x.shape[0],num_hiddens,device)
inputs=to_onehot(x.to(device),vocab_size)
params=get_params()
outputs,state_new=gru(inputs,state,params)
print(len(outputs),outputs[0].shape,state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


In [42]:
predict_rnn('分开',10,gru,params,init_gru_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'分开还妥滩轮枚永绪坦错飞'

In [43]:
def grad_clipping(params,theta,device):
    norm=torch.Tensor([0.0])
    for p in params:
        norm +=(p.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm>theta:
        for p in params:
            p.grad.data *= (theta/norm)

In [44]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad /batch_size

In [52]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,corpus_indices,idx_to_char,char_to_idx,is_random_iter,
                          num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
    if is_random_iter:
        data_iter_fn=data_iter_random
    else:
        data_iter_fn=data_iter_consecutive
    params=get_params()
    loss=nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state=init_gru_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps)
        for x,y in data_iter:
            if is_random_iter:
                state=init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
            inputs=to_onehot(x,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            y=torch.transpose(y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            
            if params[0].grad is not None:
                for p in params:
                    p.grad.data.zero_()
            l.backward()
            grad_clipping(params,clipping_theta,device)
            sgd(params,lr,1)
            l_sum+=l.item() * y.shape[0]
            n+=y.shape[0]
            
            if (epoch+1) % pred_period ==0:
                print('epoch %d ,perplexity %f,time %.2f sec' % (epoch +1,math.exp(l_sum /n),time.time()-start))
                for prefix in prefixes:
                    print('-',predict_rnn(prefix,pred_len,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx))
            

In [53]:
num_epochs,num_steps,batch_size,lr,clipping_theta=160,35,32,1e2,1e-2
pred_period,pred_len,prefixes=40,20,['分开','不分开']

In [ ]:
train_and_predict_rnn(gru,get_params,init_gru_state,num_hiddens,vocab_size,
                     corpus_indices,idx_to_char,char_to_idx,None,
                     num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,
                     pred_len,prefixes)

epoch 40 ,perplexity 121.872994,time 0.25 sec
- 分开 我不                 
- 不分开 我想                 
epoch 40 ,perplexity 112.650825,time 0.58 sec
- 分开 我想的让我 我想要你的可爱女人 坏坏的
- 不分开 我想的让我疯的可爱女人 坏坏的让我疯的
epoch 40 ,perplexity 112.224849,time 0.87 sec
- 分开 我想你 我想你 我想你 我想你 我想你
- 不分开 我想你 我想你 我想你 我想你 我想你
epoch 40 ,perplexity 112.418756,time 1.14 sec
- 分开 我想你的让我 我想你的可爱人  坏的让
- 不分开 我想你的让我 我想你的可爱人  坏的让
epoch 40 ,perplexity 112.955479,time 1.44 sec
- 分开 我想你 我想你 你不觉 我想想 你不觉
- 不分开 我想你 你不觉 我想想 你不觉 我想想
epoch 40 ,perplexity 111.266825,time 1.75 sec
- 分开 我不的让我爱女人  坏的让我疯的可爱女
- 不分开 我想你的让我爱女人  坏的让我疯的可爱
epoch 40 ,perplexity 114.238208,time 2.04 sec
- 分开 我不要 我不要 我不要 我想要你 我想
- 不分开 我想你的可爱爱人 我想你的可爱爱女人 
epoch 40 ,perplexity 114.830200,time 2.34 sec
- 分开 我想你的可爱人 我想想你 我想你 我不
- 不分开 我想你的可爱女人 我想想你 我不要 我
